# Citation Network Analysis
<br><br>
In this notebook we are going to explore the citation analysis in “Climate Change Mitigation” using Scopus API in
the last five years.<br> This notebook is part of Social Network analysis course project.